In [1]:
#imports

import os
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from collections import Counter

### Step1 : Cleaning and Preparing the data for the model

#### Here we removed the un-necessary words, expressions and symbols from the text i.e. we remove the stop words.

In [29]:
#function to create a dictionary

def makeDictionary(root_file): #takes the path of the directory as argument
    allwords=[]
    
    emails=[os.path.join(root_file,f) for f in os.listdir(root_file)]
    for mails in emails:
        with open(mails) as mail:
            for line in mail:
                words=line.split()
                allwords+=words
    
    #this function <Counter()> is used to count hashable objects
    #implicitly creates a hash table of an iterable when invoked
    dictionary= Counter(allwords) 
    #print("dictionary=",dictionary)
    
    listtoremove=dictionary.keys()
    #print("listtoremove=",listtoremove)
    
    for item in list(listtoremove):
        if item.isalpha()==False: #checks if all the characters are alphabet or not
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
            
    dictionary=dictionary.most_common(3000)
    #print("most common ,dictionary=",dictionary)
    
    return dictionary

### Step2 : Extracting Features and Corresponding Label Matrix

#### With the help of the dictionary --> generate a label and word frequency matrix


In [34]:
def extractfeatures(mail_dir):
    files=[os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix=np.zeros((len(files),3000))
    train_labels=np.zeros(len(files))
    
    count=0
    docId=0
    
    for file in files:
        with open(file) as f:
            for i, line in enumerate(f):
                if i==2:
                    words=line.split()
                    for word in words:
                        wordID=0
                        for i,d in enumerate(dictionary):
                            if d[0]==word:
                                wordID=i
                                features_matrix[docId,wordID]=words.count(word)
            train_labels[docId]=0
            filepathTokens=file.split("\\")
            lastToken=filepathTokens[len(filepathTokens)-1]
            if lastToken.startswith("spmsg"):
                train_labels[docId]=1
                count=count+1
            docId=docId+1
        return features_matrix,train_labels

### Step3 : Using Sklearn Naive Bayes--> Train and Predict

#### The Gaussian Model is used here for training.
##### It assumes that the features follow a normal distribution.

In [19]:
TRAIN_DIR = os.getcwd()+"\\train-mails"
TEST_DIR = os.getcwd()+"\\test-mails"

In [36]:
dictionary = makeDictionary(TRAIN_DIR)
print ("reading and processing emails from file")
features_matrix, labels = extractfeatures(TRAIN_DIR)
test_feature_matrix, test_labels = extractfeatures(TEST_DIR)

reading and processing emails from file


In [42]:
model = GaussianNB()

#train model
print ("--------Training model---------")
model.fit(features_matrix, labels)

predicted_labels = model.predict(test_feature_matrix)

--------Training model---------


### Step4 : Calculating the Accuracy Score
##### Accuracy score is just the percentage of correct predictions.
Sklearn provides a neat implementation for accuaracy score calculation.

In [43]:
print ("Finished classifying. \naccuracy score : ",accuracy_score(test_labels, predicted_labels))

Finished classifying. 
accuracy score :  1.0
